In [1]:
import numpy as np

In [2]:
# set up
n = 1024
a_shape = (n, n)
b_shape = (n, n)
dtype = np.float32

# Numpy

In [3]:
a = np.random.uniform(-1, 1, size=a_shape).astype(dtype)
b = np.random.uniform(-1, 1, size=b_shape).astype(dtype)
o = np.empty((a_shape[0], b_shape[1]), dtype=dtype)

%timeit c = np.dot(a, b, out=o)

19.2 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## OpenCL Preparation

In [4]:
import pyopencl as cl
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)
ctx, queue

(<pyopencl.Context at 0x26c62833800 on <pyopencl.Device 'GeForce GPU' on 'NVIDIA CUDA' at 0x26c632950b0>>,
 <pyopencl._cl.CommandQueue at 0x26c64190d08>)

In [5]:
# create inputs
a = np.random.uniform(-1, 1, size=a_shape).astype(dtype)
b = np.random.uniform(-1, 1, size=b_shape).astype(dtype)
a_buf = cl.Buffer(ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=a)
b_buf = cl.Buffer(ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=b)
# create output buffer
o = np.empty((a_shape[0], b_shape[1]), dtype=dtype)
o_buf = cl.Buffer(ctx, cl.mem_flags.READ_WRITE, size=o.dtype.itemsize * a.shape[0] * b.shape[1])
a_buf, b_buf, o_buf

(<pyopencl._cl.Buffer at 0x26c641bdfa8>,
 <pyopencl._cl.Buffer at 0x26c641907c8>)

## OpenCL Naive
Unbelievably slow (5x slower than numpy wtf)

In [6]:
source = """
// First naive implementation
__kernel void myGEMM1(const int M, const int N, const int K,
                      const __global float* A,
                      const __global float* B,
                      __global float* C) {
    
    // Thread identifiers
    const int globalRow = get_global_id(0); // Row ID of C (0..M)
    const int globalCol = get_global_id(1); // Col ID of C (0..N)
 
    // Compute a single element (loop over K)
    float acc = 0.0f;
    for (int k=0; k<K; k++) {
        acc += A[k*M + globalRow] * B[globalCol*K + k];
    }
 
    // Store the result
    C[globalCol*M + globalRow] = acc;
}
"""
prg = cl.Program(ctx, source).build()

In [7]:
m, n, k = a_shape[0], b_shape[1], b_shape[0]
# check correctness
prg.myGEMM1(queue, o.shape, None, np.int32(m), np.int32(n), np.int32(k), b_buf, a_buf, o_buf)
cl.enqueue_copy(queue, o, o_buf)
queue.flush()

d = np.abs(o - (a @ b))
np.linalg.norm(d), np.max(d)

(0.0065565286, 8.010864e-05)

In [8]:
# timeit
def matmul_opencl():
    prg.myGEMM1(queue, o.shape, None, np.int32(m), np.int32(n), np.int32(k), b_buf, a_buf, o_buf)
    queue.finish()
matmul_opencl()

%timeit -n 100 matmul_opencl()

100 ms ± 58.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# OpenCL using local Blocks
Pretty much as fast as numpy

In [9]:
source = """
// from: https://github.com/sschaetz/nvidia-opencl-examples/blob/master/OpenCL/src/oclMultiThreads/matrixMul.cl

#define BLOCK_SIZE 16
#define AS(i, j) As[j + i * BLOCK_SIZE]
#define BS(i, j) Bs[j + i * BLOCK_SIZE]

///////////////////////////////////////////////////////////////////////////////
//! Matrix multiplication on the device: C = A * B
//! uiWA is A's width and uiWB is B's width
////////////////////////////////////////////////////////////////////////////////
__kernel void
matrixMul( __global float* C, __global float* A, __global float* B, int uiWA, int uiWB)
{

    // allocate local memory
    __local float As[BLOCK_SIZE * BLOCK_SIZE];
    __local float Bs[BLOCK_SIZE * BLOCK_SIZE];

    // Block index
    int bx = get_group_id(0);
    int by = get_group_id(1);

    // Thread index
    int tx = get_local_id(0);
    int ty = get_local_id(1);

    // Index of the first sub-matrix of A processed by the block
    int aBegin = uiWA * BLOCK_SIZE * by;

    // Index of the last sub-matrix of A processed by the block
    int aEnd   = aBegin + uiWA - 1;

    // Step size used to iterate through the sub-matrices of A
    int aStep  = BLOCK_SIZE;

    // Index of the first sub-matrix of B processed by the block
    int bBegin = BLOCK_SIZE * bx;

    // Step size used to iterate through the sub-matrices of B
    int bStep  = BLOCK_SIZE * uiWB;

    // Csub is used to store the element of the block sub-matrix
    // that is computed by the thread
    float Csub = 0.0f;

    // Loop over all the sub-matrices of A and B
    // required to compute the block sub-matrix
    for (int a = aBegin, b = bBegin;
             a <= aEnd;
             a += aStep, b += bStep) {

        // Load the matrices from device memory
        // to shared memory; each thread loads
        // one element of each matrix
        AS(ty, tx) = A[a + uiWA * ty + tx];
        BS(ty, tx) = B[b + uiWB * ty + tx];
	
        // Synchronize to make sure the matrices are loaded
        barrier(CLK_LOCAL_MEM_FENCE);

        // Multiply the two matrices together;
        // each thread computes one element
        // of the block sub-matrix        
        #pragma unroll
        for (int k = 0; k < BLOCK_SIZE; ++k)
            Csub += AS(ty, k) * BS(k, tx);

        // Synchronize to make sure that the preceding
        // computation is done before loading two new
        // sub-matrices of A and B in the next iteration
        barrier(CLK_LOCAL_MEM_FENCE);
    }

    // Write the block sub-matrix to device memory;
    // each thread writes one element
    C[get_global_id(1) * get_global_size(0) + get_global_id(0)] = Csub;

}
"""

prg = cl.Program(ctx, source).build()

In [10]:
# check correctness
prg.matrixMul(queue, o.shape, [16, 16], o_buf, a_buf, b_buf, np.int32(a_shape[1]), np.int32(b_shape[1]))
cl.enqueue_copy(queue, o, o_buf)
queue.flush()

d = np.abs(o - (a @ b))
np.linalg.norm(d), np.max(d)

(0.0065565286, 8.010864e-05)

In [11]:
# timeit
def matmul_opencl():
    prg.matrixMul(queue, o.shape, [16, 16], o_buf, a_buf, b_buf, np.int32(a_shape[1]), np.int32(b_shape[1]))
    queue.finish()
matmul_opencl()

%timeit -n 100 matmul_opencl()

23 ms ± 35.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# OpenCL using Blocks and Registers
Finally something that beats numpy

In [12]:
source = """
// from: https://cnugteren.github.io/tutorial/pages/page8.html

#define TSM 128                // The tile-size in dimension M
#define TSN 128                // The tile-size in dimension N
#define TSK 16                 // The tile-size in dimension K
#define WPTM 8                 // The work-per-thread in dimension M
#define WPTN 8                 // The work-per-thread in dimension N
#define RTSM (TSM/WPTM)        // The reduced tile-size in dimension M
#define RTSN (TSN/WPTN)        // The reduced tile-size in dimension N
#define LPTA ((TSK*TSM)/(RTSM*RTSN)) // Loads-per-thread for A
#define LPTB ((TSK*TSN)/(RTSM*RTSN)) // Loads-per-thread for B

// Use 2D register blocking (further increase in work per thread)
__kernel void myGEMM6(const int M, const int N, const int K,
                      const __global float* A,
                      const __global float* B,
                      __global float* C) {
    
    // Thread identifiers
    const int tidm = get_local_id(0); // Local row ID (max: TSM/WPTM)
    const int tidn = get_local_id(1); // Local col ID (max: TSN/WPTN)
    const int offsetM = TSM*get_group_id(0); // Work-group offset
    const int offsetN = TSN*get_group_id(1); // Work-group offset
 
    // Local memory to fit a tile of A and B
    __local float Asub[TSK][TSM];
    __local float Bsub[TSN][TSK];
 
    // Allocate register space
    float Areg;
    float Breg[WPTN];
    float acc[WPTM][WPTN];
 
    // Initialise the accumulation registers
    for (int wm=0; wm<WPTM; wm++) {
        for (int wn=0; wn<WPTN; wn++) {
            acc[wm][wn] = 0.0f;
        }
    }
        
    // Loop over all tiles
    int numTiles = K/TSK;
    for (int t=0; t<numTiles; t++) {
 
        // Load one tile of A and B into local memory
        for (int la=0; la<LPTA; la++) {
            int tid = tidn*RTSM + tidm;
            int id = la*RTSN*RTSM + tid;
            int row = id % TSM;
            int col = id / TSM;
            int tiledIndex = TSK*t + col;
            Asub[col][row] = A[tiledIndex*M + offsetM + row];
            Bsub[row][col] = B[tiledIndex*N + offsetN + row];
            // use transposed matrix B
            // Bsub[row][col] = B[tiledIndex + (offsetN + row) * K];
        }
        
        // Synchronise to make sure the tile is loaded
        barrier(CLK_LOCAL_MEM_FENCE);
 
        // Loop over the values of a single tile
        for (int k=0; k<TSK; k++) {
 
            // Cache the values of Bsub in registers
            for (int wn=0; wn<WPTN; wn++) {
                int col = tidn + wn*RTSN;
                Breg[wn] = Bsub[col][k];
            }
 
            // Perform the computation
            for (int wm=0; wm<WPTM; wm++) {
                int row = tidm + wm*RTSM;
                Areg = Asub[k][row];
                for (int wn=0; wn<WPTN; wn++) {
                    acc[wm][wn] += Areg * Breg[wn];
                }
            }
        }
 
        // Synchronise before loading the next tile
        barrier(CLK_LOCAL_MEM_FENCE);
    }
 
    // Store the final results in C
    for (int wm=0; wm<WPTM; wm++) {
        int globalRow = offsetM + tidm + wm*RTSM;
        for (int wn=0; wn<WPTN; wn++) {
            int globalCol = offsetN + tidn + wn*RTSN;
            C[globalCol*M + globalRow] = acc[wm][wn];
        }
    }
}
"""

prg = cl.Program(ctx, source).build()

In [13]:
m, n, k = a_shape[0], b_shape[1], b_shape[0]

g_shape = (m//8, n//8)
l_shape = (128//8, 128//8)

# check correctness - notice that a and b positions are swapped
prg.myGEMM6(queue, g_shape, l_shape, np.int32(m), np.int32(n), np.int32(k), b_buf, a_buf, o_buf)
cl.enqueue_copy(queue, o, o_buf)
queue.flush()

# optimized memory accesses -> weird transposition of matrices
d = np.abs(o - (a.T @ b))  # this is faster because ordered memory accesses (~5.86ms)
# d = np.abs(o - (a @ b))      # when b is transposed in opencl code (~6.7ms)
np.linalg.norm(d), np.max(d)

(0.00655901, 8.010864e-05)

In [14]:
# timeit
def matmul_opencl():
    prg.myGEMM6(queue, g_shape, l_shape, np.int32(m), np.int32(n), np.int32(k), b_buf, a_buf, o_buf)
    queue.finish()
matmul_opencl()

%timeit -n 100 matmul_opencl()

5.87 ms ± 68.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
